# Projet : Exploration RAG et RLHF

**Binôme 7** :
**EL GHIOUAN Israe & BENCHERAIK ABDESSAMAD**

Lien Github : https://github.com/elghiouan/Projet2-DeepLearningLab/

Ce notebook démontre :
1.  **Génération Augmentée par Récupération (RAG) :** Construction d'un système qui répond à des questions basées sur une base de connaissances fournie en utilisant LangChain et les modèles Hugging Face.
2.  **Apprentissage par Renforcement à partir de Retours Humains (RLHF) :** Un aperçu conceptuel et une démonstration de code simplifiée de la manière dont RLHF peut être utilisé pour aligner les modèles de langage, en utilisant la bibliothèque TRL.

Nous utiliserons Google Colab avec un GPU T4.

In [ ]:
# Installation des dépendances et connexion à Hugging Face
!pip install -q --upgrade pip
!pip install -q \
    "fsspec==2025.3.2" \
    "gcsfs>=2025.3.2" \
    transformers \
    langchain \
    langchain-community \
    sentence-transformers \
    faiss-cpu \
    accelerate \
    bitsandbytes \
    huggingface_hub \
    datasets \
    trl \
    einops

from huggingface_hub import notebook_login
notebook_login()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 133.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 123.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# Importations principales
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig, AutoModelForSeq2SeqLM
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document
from datasets import load_dataset
import os
import requests
import json
from trl import PPOConfig, PPOTrainer, AutoModelForCausalLMWithValueHead
import numpy as np
import sys

## Partie 1 : Génération Augmentée par Récupération (RAG)

La RAG améliore les LLM en leur fournissant des connaissances externes.
Le processus implique typiquement :
1.  **Indexation :** Création d'une base de données vectorielle interrogeable.
2.  **Récupération :** Les documents pertinents sont récupérés de la base.
3.  **Génération :** Le LLM utilise la requête et les documents récupérés pour répondre.

Nous utilisons SQuAD comme source de connaissances.

In [ ]:
# Vérification du GPU et chargement du modèle pour RAG
if torch.cuda.is_available():
    print(f"Informations GPU : Nom - {torch.cuda.get_device_name(0)}, Mémoire Totale - {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    torch.cuda.empty_cache()
    print(f"Mémoire GPU Disponible (après vidage) : {torch.cuda.mem_get_info()[0] / 1e9:.2f} GB")
else:
    print("AVERTISSEMENT : CUDA n'est pas disponible.")

model_id_rag = "google/flan-t5-large"
print(f"Chargement du modèle RAG : {model_id_rag}")
tokenizer_rag = AutoTokenizer.from_pretrained(model_id_rag)
device_map_config = {"": 0} if torch.cuda.is_available() else "auto"
model_kwargs_rag = {"device_map": device_map_config, "torch_dtype": torch.bfloat16 if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else torch.float32}
model_rag = AutoModelForSeq2SeqLM.from_pretrained(model_id_rag, **model_kwargs_rag)

text_generation_pipeline_rag = pipeline(
    "text2text-generation",
    model=model_rag,
    tokenizer=tokenizer_rag,
    max_new_tokens=150,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1
)
llm_rag = HuggingFacePipeline(pipeline=text_generation_pipeline_rag)
print("LLM pour RAG chargé et Pipeline Créé.")

# Préparation des données SQuAD pour RAG
squad_train_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json"
squad_train_filename = "squad_train-v1.1.json"
squad_dataset_raw_list = []

if not os.path.exists(squad_train_filename):
    print(f"Téléchargement de {squad_train_filename}...")
    try:
        response = requests.get(squad_train_url, stream=True)
        response.raise_for_status()
        with open(squad_train_filename, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"{squad_train_filename} téléchargé.")
    except requests.exceptions.RequestException as e:
        print(f"Erreur de téléchargement {squad_train_filename}: {e}")
        squad_train_filename = None
else:
    print(f"{squad_train_filename} existe déjà.")

if squad_train_filename and os.path.exists(squad_train_filename):
    print("Analyse manuelle du JSON SQuAD...")
    try:
        with open(squad_train_filename, 'r', encoding='utf-8') as f:
            squad_data_content = json.load(f)
        for article in squad_data_content.get('data', []):
            for paragraph in article.get('paragraphs', []):
                context = paragraph.get('context')
                if context: # S'assurer que le contexte existe
                    for qa in paragraph.get('qas', []):
                        question = qa.get('question')
                        answer_texts = [ans.get('text') for ans in qa.get('answers', []) if ans.get('text') is not None]
                        if question and answer_texts:
                            squad_dataset_raw_list.append({
                                "title": article.get('title', 'Titre N/A'),
                                "context": context,
                                "question": question,
                                "answers": {"text": answer_texts, "answer_start": [ans.get('answer_start', -1) for ans in qa.get('answers', [])]}
                            })
        squad_dataset_raw_list = squad_dataset_raw_list[:5000] # Limiter pour la démo
        print(f"{len(squad_dataset_raw_list)} exemples parsés depuis SQuAD JSON.")
    except Exception as e:
        print(f"Erreur d'analyse JSON : {e}")
        squad_dataset_raw_list = []
else:
    print("Fichier SQuAD non disponible pour l'analyse.")

if not squad_dataset_raw_list: # Tentative de chargement direct si l'analyse manuelle a échoué
    print("Tentative de chargement direct de SQuAD via datasets...")
    try:
        squad_dataset_raw_direct = load_dataset("squad", name="plain_text", split="train[:5000]", trust_remote_code=True)
        squad_dataset_raw_list = [example for example in squad_dataset_raw_direct]
        print(f"{len(squad_dataset_raw_list)} exemples chargés via load_dataset.")
    except Exception as e_direct:
        print(f"Échec du chargement direct : {e_direct}")

if squad_dataset_raw_list:
    unique_contexts_rag = {}
    for example in squad_dataset_raw_list:
        context = example.get('context')
        if context and context not in unique_contexts_rag:
            unique_contexts_rag[context] = {
                "title": example.get('title', 'N/A'),
                "first_question": example.get('question', 'N/A'),
                "first_answer": example.get('answers', {}).get('text', [])[0] if example.get('answers', {}).get('text') else "N/A"
            }
    documents_rag = [Document(page_content=ctxt, metadata=meta) for i, (ctxt, meta) in enumerate(unique_contexts_rag.items())]
    # Mettre à jour les métadonnées pour inclure doc_id
    for i, doc in enumerate(documents_rag):
        doc.metadata["doc_id"] = i
    print(f"{len(documents_rag)} documents uniques préparés.")
else:
    print("Échec du chargement des données SQuAD.")
    documents_rag = [] # S'assurer que documents_rag est défini

Informations GPU : Nom - Tesla T4, Mémoire Totale - 15.83 GB
Mémoire GPU Disponible (après vidage) : 13.87 GB
Chargement du modèle RAG : google/flan-t5-large


Device set to use cuda:0


LLM pour RAG chargé et Pipeline Créé.
squad_train-v1.1.json existe déjà.
Analyse manuelle du JSON SQuAD...
5000 exemples parsés depuis SQuAD JSON.
820 documents uniques préparés.


In [ ]:
# Création des Embeddings, Vector Store et Chaîne RAG
if documents_rag: # S'assurer que documents_rag n'est pas vide
    embedding_model_name_rag = "sentence-transformers/all-MiniLM-L6-v2"
    embedding_model_rag = HuggingFaceEmbeddings(model_name=embedding_model_name_rag)
    print("Création de la base vectorielle (FAISS)...")
    vectorstore_rag = FAISS.from_documents(documents_rag, embedding_model_rag)
    retriever_rag = vectorstore_rag.as_retriever(search_kwargs={"k": 3})
    print("Base vectorielle et Retriever créés.")

    qa_chain_rag = RetrievalQA.from_chain_type(
        llm=llm_rag,
        chain_type="stuff",
        retriever=retriever_rag,
        return_source_documents=True
    )
    print("Chaîne QA RAG créée.")
else:
    print("Aucun document chargé, la création de la base vectorielle et de la chaîne RAG est ignorée.")
    vectorstore_rag = None
    retriever_rag = None
    qa_chain_rag = None

<ipython-input-6-7128880dcd20>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model_rag = HuggingFaceEmbeddings(model_name=embedding_model_name_rag)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Création de la base vectorielle (FAISS)...
Base vectorielle et Retriever créés.
Chaîne QA RAG créée.


In [ ]:
# Test de la chaîne RAG
if qa_chain_rag and documents_rag:
    doc_index_for_test = 0
    if len(documents_rag) > doc_index_for_test:
        # Utilisation de .get() pour éviter les KeyError si les clés n'existent pas
        test_question_rag = documents_rag[doc_index_for_test].metadata.get("sample_question", "Quelle est la capitale de la France ?")
        expected_answer_rag = documents_rag[doc_index_for_test].metadata.get("sample_answer", "Paris")
    else:
        test_question_rag = "Quelle est une utilisation courante de l'IA ?"
        expected_answer_rag = "Diverses applications"

    print(f"\nQuestion Test : {test_question_rag}")
    print(f"(Réponse attendue du contexte : {expected_answer_rag})")
    response_rag = qa_chain_rag.invoke({"query": test_question_rag})
    print("\nRéponse Générée par RAG :")
    print(response_rag["result"])
    print("\nDocuments Sources Récupérés :")
    if response_rag.get("source_documents"):
        for i, doc in enumerate(response_rag["source_documents"]):
            print(f"--- Document {i+1} (Titre Source : {doc.metadata.get('source_title', 'N/A')}) ---")
            print(doc.page_content[:250] + "...")
else:
    print("Test RAG ignoré : chaîne ou documents non disponibles.")


Question Test : Quelle est la capitale de la France ?
(Réponse attendue du contexte : Paris)

Réponse Générée par RAG :
Lourdes

Documents Sources Récupérés :
--- Document 1 (Titre Source : N/A) ---
The University of Notre Dame du Lac (or simply Notre Dame /ˌnoʊtərˈdeɪm/ NOH-tər-DAYM) is a Catholic research university located adjacent to South Bend, Indiana, in the United States. In French, Notre Dame du Lac means "Our Lady of the Lake" and refe...
--- Document 2 (Titre Source : N/A) ---
The first documented visit by a European was in 1524 by Giovanni da Verrazzano, a Florentine explorer in the service of the French crown, who sailed his ship La Dauphine into New York Harbor. He claimed the area for France and named it "Nouvelle Ango...
--- Document 3 (Titre Source : N/A) ---
Because of its Catholic identity, a number of religious buildings stand on campus. The Old College building has become one of two seminaries on campus run by the Congregation of Holy Cross. The current Basilica o

## Partie 2 : RLHF (Apprentissage par Renforcement à partir de Retours Humains) - Aperçu Conceptuel

RLHF aligne les LLM avec les préférences humaines.
Étapes typiques :
1.  **SFT :** Ajustement fin supervisé (non couvert ici).
2.  **Entraînement du Modèle de Récompense (RM) :** Apprend à prédire les préférences humaines.
3.  **Ajustement Fin par RL (PPO) :** Le LLM apprend à maximiser les scores du RM.

La bibliothèque `trl` simplifie l'étape PPO.
**Remarque :** Le code suivant est une **démonstration conceptuelle** utilisant une fonction de récompense fictive.

In [ ]:
# Configuration PPO, chargement du modèle et données fictives pour RLHF
ppo_config_args = {
    "batch_size": 1, "mini_batch_size": 1, "gradient_accumulation_steps": 1,
    "learning_rate": 1.5e-5, "adap_kl_ctrl": True, "init_kl_coef": 0.05,
    "target_kl": 0.2, "log_with": None, "ppo_epochs": 2, "seed": 42,
}
ppo_config = PPOConfig(**ppo_config_args)
print("PPOConfig créé.")

base_model_id_ppo = "gpt2" # Modèle plus petit pour la démo
print(f"Chargement du modèle de base pour PPO : {base_model_id_ppo}")
ppo_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ppo_model = AutoModelForCausalLMWithValueHead.from_pretrained(base_model_id_ppo).to(ppo_device)
ppo_tokenizer = AutoTokenizer.from_pretrained(base_model_id_ppo)
if ppo_tokenizer.pad_token is None:
    ppo_tokenizer.pad_token = ppo_tokenizer.eos_token
    ppo_model.config.pad_token_id = ppo_tokenizer.eos_token_id
print(f"Modèle PPO '{base_model_id_ppo}' chargé sur : {ppo_device}.")

dummy_prompts_text_ppo = [
    "Expliquez la théorie de la relativité en termes simples.",
    "Écrivez un court poème sur un chat curieux.",
    "Quels sont les trois avantages de l'exercice régulier ?",
]
dummy_prompts_tokenized_ppo = [ppo_tokenizer.encode(prompt, return_tensors="pt").to(ppo_device).squeeze(0) for prompt in dummy_prompts_text_ppo]
print(f"{len(dummy_prompts_tokenized_ppo)} invites fictives pour PPO.")

# Fonction de récompense FICTIVE. Ne pas utiliser en production.
def dummy_reward_function_ppo(texts_list):
    rewards = []
    for text in texts_list:
        score = 0.0
        if "relativité" in text.lower(): score += 0.8
        elif "chat" in text.lower() and "poème" in text.lower(): score += 0.7
        elif "exercice" in text.lower() and "avantages" in text.lower(): score += 0.9
        if 30 < len(text) < 150: score += 0.2
        elif len(text) >= 150: score -= 0.1
        if "je ne sais pas" in text.lower(): score -= 1.5
        rewards.append(torch.tensor(score, device=ppo_device, dtype=torch.float32))
    return rewards
print("Fonction de récompense FICTIVE définie.")

try:
    ppo_trainer = PPOTrainer(
        config=ppo_config,
        model=ppo_model,
        ref_model=None,
        tokenizer=ppo_tokenizer,
    )
    print("PPOTrainer Initialisé.")
except Exception as e:
    print(f"Erreur d'initialisation PPOTrainer : {e}")
    ppo_trainer = None

/usr/local/lib/python3.11/dist-packages/trl/trainer/ppo_config.py:207: FutureWarning: `PPOConfig` is deprecated and will be removed in the future. Please use `PPOv2Config` with `PPOv2Trainer` instead.
  warnings.warn(


PPOConfig créé.
Chargement du modèle de base pour PPO : gpt2


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Modèle PPO 'gpt2' chargé sur : cuda.
3 invites fictives pour PPO.
Fonction de récompense FICTIVE définie.
PPOTrainer Initialisé.


/usr/local/lib/python3.11/dist-packages/trl/trainer/ppo_trainer.py:193: FutureWarning: `PPOTrainer` is deprecated and will be removed in trl v0.12. Please use `PPOv2Trainer` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/ppo_trainer.py:273: UserWarning: No dataset is provided. Make sure to set config.batch_size to the correct value before training.
  warnings.warn(


In [ ]:
# Boucle d'entraînement PPO conceptuelle
if ppo_trainer:
    print("\nBoucle d'Entraînement PPO Conceptuelle :")
    generation_kwargs_ppo = {
        "min_length": -1, "top_k": 0.0, "top_p": 1.0, "do_sample": True,
        "pad_token_id": ppo_tokenizer.pad_token_id, "max_new_tokens": 70,
        "eos_token_id": ppo_tokenizer.eos_token_id,
    }
    max_ppo_steps = len(dummy_prompts_tokenized_ppo)

    for step in range(max_ppo_steps):
        current_prompt_tensor = dummy_prompts_tokenized_ppo[step]
        current_prompt_text = dummy_prompts_text_ppo[step]
        print(f"\n--- Étape {step+1}/{max_ppo_steps} --- Invite : \"{current_prompt_text}\"")

        query_tensors_batch = [current_prompt_tensor.to(ppo_device)]
        ppo_model.to(ppo_device)
        response_tensors_batch = ppo_trainer.generate(query_tensors_batch, **generation_kwargs_ppo)
        full_texts_batch = [ppo_tokenizer.decode(r.squeeze(), skip_special_tokens=True) for r in response_tensors_batch]
        print(f"Réponse Générée : \"{full_texts_batch[0]}\"")

        rewards_batch = dummy_reward_function_ppo(full_texts_batch)
        stats = ppo_trainer.step(query_tensors_batch, response_tensors_batch, rewards_batch)
        mean_reward = torch.mean(torch.stack(rewards_batch)).item()

        kl_value_from_stats = stats.get('objective/kl', 0.0) # Valeur par défaut si non trouvée
        kl_div = kl_value_from_stats.item() if isinstance(kl_value_from_stats, torch.Tensor) else float(kl_value_from_stats)
        print(f"Statistiques : Divergence KL : {kl_div:.3f}, Récompense Moyenne (fictive) : {mean_reward:.3f}")

    print("\nBoucle d'entraînement PPO conceptuelle terminée.")
else:
    print("Boucle PPO conceptuelle ignorée : PPOTrainer non initialisé.")

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



Boucle d'Entraînement PPO Conceptuelle :

--- Étape 1/3 --- Invite : "Expliquez la théorie de la relativité en termes simples."
Réponse Générée : "Expliquez la théorie de la relativité en termes simples. He voyageurs protagonistlon tant, nostalgiate d'objetor loclon pour sous la Nation des Grenoble Powers, entendu à mere est politique classique semaine que cette se rappinaire madame peuple, à l'expression en la universalité, communauté otheréness économique"


/usr/local/lib/python3.11/dist-packages/trl/trainer/ppo_trainer.py:1309: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  std_scores = data["scores"].std()
/usr/local/lib/python3.11/dist-packages/trl/trainer/ppo_trainer.py:1336: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  stats["tokens/queries_len_std"] = torch.std(query_lens).cpu().numpy().item()
/usr/local/lib/python3.11/dist-packages/trl/trainer/ppo_trainer.py:1339: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1

Statistiques : Divergence KL : 0.000, Récompense Moyenne (fictive) : 0.700

--- Étape 2/3 --- Invite : "Écrivez un court poème sur un chat curieux."
Réponse Générée : "Écrivez un court poème sur un chat curieux. Champagne 2007 - 1Number 130 STA CHAMPAGE CHAMPAGE

Santos arguing Stockholm, Germany (= Kosovo Defender)

ztana barred has 1966 European Union of European treaties and for that matter the SAA has 38 allied treaty comesson cancelled 1995-10-51 SGS 1980 vs 1965 EUAC 1936 vs 1971"


/usr/local/lib/python3.11/dist-packages/trl/trainer/ppo_trainer.py:1313: UserWarning: KL divergence is starting to become negative: -4.25 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(


Statistiques : Divergence KL : -4.246, Récompense Moyenne (fictive) : 0.600

--- Étape 3/3 --- Invite : "Quels sont les trois avantages de l'exercice régulier ?"
Réponse Générée : "Quels sont les trois avantages de l'exercice régulier ? Liège avec je s'Liège en efférence.

Hérant vu sadais * LET Brunhmain annaire plus ans assisted Servier hin de ouvre la side »

info de Tour de Nobles et Estretux Arts. 89 dis les décisions, sont"
Statistiques : Divergence KL : -4.305, Récompense Moyenne (fictive) : 0.800

Boucle d'entraînement PPO conceptuelle terminée.


/usr/local/lib/python3.11/dist-packages/trl/trainer/ppo_trainer.py:1313: UserWarning: KL divergence is starting to become negative: -4.31 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(


In [ ]:
# Session Q&A interactive avec RAG
print("\n--- Session Q&A Interactive avec RAG ---")
print("Posez une question (tapez 'quittez', 'exit', ou 'sortir' pour terminer).")

if 'qa_chain_rag' in globals() and qa_chain_rag is not None:
    while True:
        try:
            user_question = input("\nVotre question : ")
            if user_question.strip().lower() in ["quittez", "exit", "sortir", "q"]:
                print("Au revoir !")
                break
            if not user_question.strip():
                print("Veuillez entrer une question.")
                continue

            print("Recherche de la réponse...")
            response = qa_chain_rag.invoke({"query": user_question})
            print("\nRéponse du système RAG :")
            print(response.get("result", "Désolé, réponse non trouvée."))

            if response.get("source_documents"):
                print("\nDocuments sources consultés :")
                for i, doc in enumerate(response["source_documents"]):
                    source_title = doc.metadata.get('source_title', 'N/A')
                    content_snippet = str(doc.page_content)[:200] + "..." if hasattr(doc, 'page_content') and isinstance(doc.page_content, str) else "Contenu non disponible."
                    print(f"--- Document {i+1} (Source : {source_title}) ---")
                    print(content_snippet)
        except KeyboardInterrupt:
            print("\nInteraction interrompue. Au revoir !")
            break
        except Exception as e:
            print(f"Une erreur est survenue : {e}")
else:
    print("Le système RAG (qa_chain_rag) n'a pas été initialisé correctement.")


--- Session Q&A Interactive avec RAG ---
Posez une question (tapez 'quittez', 'exit', ou 'sortir' pour terminer).
Recherche de la réponse...

Réponse du système RAG :
Solar radiation

Documents sources consultés :
--- Document 1 (Source : N/A) ---
Solar radiation is absorbed by the Earth's land surface, oceans – which cover about 71% of the globe – and atmosphere. Warm air containing evaporated water from the oceans rises, causing atmospheric c...
--- Document 2 (Source : N/A) ---
Solar technologies are broadly characterized as either passive or active depending on the way they capture, convert and distribute sunlight and enable solar energy to be harnessed at different levels ...
--- Document 3 (Source : N/A) ---
Active solar techniques use photovoltaics, concentrated solar power, solar thermal collectors, pumps, and fans to convert sunlight into useful outputs. Passive solar techniques include selecting mater...
Recherche de la réponse...

Réponse du système RAG :
gravity

Documents

## Conclusion

Ce notebook a démontré :
1.  **RAG :** Construction d'un système de questions-réponses exploitant des documents externes (Flan-T5-Large, LangChain, FAISS).
2.  **RLHF (Conceptuel) :** Idées fondamentales de l'alignement par RLHF (démo PPO simplifiée avec `trl`).
